# [atpbar](https://github.com/alphatwirl/atpbar)

Progress bars for threading and multiprocessing tasks on terminal and Jupyter Notebook

Tai Sakuma

_atpbar_ can display multiple progress bars simultaneously growing to
show the progresses of iterations of loops in
[threading](https://docs.python.org/3/library/threading.html) or
[multiprocessing](https://docs.python.org/3/library/multiprocessing.html)
tasks. _atpbar_ can display progress bars on terminal and [Jupyter
Notebook](https://jupyter.org/). _atpbar_ can be used with
[_Mantichora_](https://github.com/alphatwirl/mantichora).

_atpbar_ started its development in 2015 as part of
[_alphatwirl_](https://github.com/alphatwirl/alphatwirl). _atpbar_
prevented physicists from terminating their running analysis codes,
which would take many hours to complete, by showing progress bars
indicating their codes were actually running. The progress bars have
saved the physicists countless hours total. _atpbar_ had been the
sub-package
[_progressbar_](https://github.com/alphatwirl/alphatwirl/tree/v0.22.0/alphatwirl/progressbar)
of alphatwirl until it became an independent package, with the name
_atpbar_, in February 2019.


*****

- **GitHub:** https://github.com/alphatwirl/atpbar
- **PyPI:** https://pypi.org/project/atpbar/

*****

- [**Quick start**](#Quick-start)
    - [Import libraries](#Import-libraries)
    - [One loop](#One-loop)
    - [Nested loops](#Nested-loops)
    - [Threading](#Threading)
    - [Multiprocessing](#Multiprocessing)
- [**Advanced features**](#Advanced-features)
    - [A `break` and an exception](#A-break-and-an-exception)
    - [Progress of starting threads and processes with progress bars](#Progress-of-starting-threads-and-processes-with-progress-bars)
- [**License**](#License)
- [**Contact**](#Contact)

*****


## Quick start

I will show here how to use atpbar by simple examples.

### Import libraries

To create simple loops in the examples, we use two python standard
libraries, [time](https://docs.python.org/3/library/time.html) and
[random](https://docs.python.org/3/library/random.html). Import the
two packages as well as `atpbar`.


In [ ]:
import time, random
from atpbar import atpbar

**Note**: import the object `atpbar` from the package `atpbar` rather
than importing the package `atpbar` itself.

### One loop

The object `atpbar` is an iterable that can wrap another iterable and
shows the progress bars for the iterations. (The idea of making the
interface iterable was inspired by
[tqdm](https://github.com/tqdm/tqdm).)

The task in the code below is to sleep for `0.0001` seconds in each
iteration of the loop. The number of the iterations of the loop is
randomly selected from between `500` and `10000`.

A progress bar will be shown by `atpbar`.

In [ ]:
n = random.randint(500, 10000)
for i in atpbar(range(n)):
    time.sleep(0.0001)

In order for `atpbar` to show a progress bar, the wrapped iterable
needs to have a length. If the length cannot be obtained by `len()`,
`atpbar` won't show a progress bar.

### Nested loops

`atpbar` can show progress bars for nested loops as in the example below.

The outer loop iterates 4 times. The inner loop is similar to the loop
in the previous example—sleeps for `0.0001` seconds. You can
optionally give the keyword argument `name` to specify the label on
the progress bar.

In [ ]:
for i in atpbar(range(4), name='outer'):
    n = random.randint(500, 10000)
    for j in atpbar(range(n), name='inner {}'.format(i)):
        time.sleep(0.0001)

The progress bars for the completed tasks move up. The progress
bars for the active tasks grow at the bottom.

### Threading

`atpbar` can show multiple progress bars for loops concurrently
iterating in different threads.

The function `run_with_threading()` in the following code shows an
example. The task to sleep for `0.0001` seconds is defined as the function
`task`. The `task` is concurrently run 5 times with `threading`.
`atpbar` can be used in any threads. Five progress bars will simultaneously grow. As a task completes, the progress bar for the task moves up. The progress bars for active tasks are at the bottom. The function `flush()` returns when the progress bars have finished updating.

In [ ]:
from atpbar import flush
import threading

def run_with_threading():
    def task(n, name):
        for i in atpbar(range(n), name=name):
            time.sleep(0.0001)
    nthreads = 5
    threads = [ ]
    for i in range(nthreads):
        name = 'thread {}'.format(i)
        n = random.randint(500, 10000)
        t = threading.Thread(target=task, args=(n, name))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    flush()

run_with_threading()

### Multiprocessing

`atpbar` can be used with `multiprocessing`. A few extra lines of code
need to be added.

The function `run_with_multiprocessing()` in the following code shows
an example. It starts four workers in subprocesses with `multiprocessing` and have
them run ten tasks. In order to use `atpbar` in a subprocess, the `reporter`, which can be
found in the main process by the function `find_reporter()`, needs to
be brought to the subprocess and registered there by the function
`register_reporter()`.

Simultaneously growing progress bars will be shown.

In [ ]:
import multiprocessing
multiprocessing.set_start_method('fork', force=True)

from atpbar import register_reporter, find_reporter, flush

def run_with_multiprocessing():
    def task(n, name):
        for i in atpbar(range(n), name=name):
            time.sleep(0.0001)
    def worker(reporter, task, queue):
        register_reporter(reporter)
        while True:
            args = queue.get()
            if args is None:
                queue.task_done()
                break
            task(*args)
            queue.task_done()
    nprocesses = 4
    ntasks = 10
    reporter = find_reporter()
    queue = multiprocessing.JoinableQueue()
    for i in range(nprocesses):
        p = multiprocessing.Process(target=worker, args=(reporter, task, queue))
        p.start()
    for i in range(ntasks):
        name = 'task {}'.format(i)
        n = random.randint(500, 10000)
        queue.put((n, name))
    for i in range(nprocesses):
        queue.put(None)
        queue.join()
    flush()

run_with_multiprocessing()

*****

## Advanced features

This section introduces a few advanced features. If you skip the examples above, need to import some modules and objects.

In [ ]:
import time, random
import threading
from atpbar import atpbar, flush

### A `break` and an exception

When the loop ends with a `break` or an exception, the progress bar stops with
the last complete iteration.

For example, the loop in the following code breaks during the 1235th iteration.

In [ ]:
for i in atpbar(range(2000)):
    if i == 1234:
        break
    time.sleep(0.0001)

Since `i` starts with `0`, when `i` is `1234`, the loop is in its 1235th
iteration. The last complete iteration is 1234. The progress bar stops at 1234.


As an example of an exception, in the following code, an exception is
thrown during the 1235th iteration.

In [ ]:
for i in atpbar(range(2000)):
    if i == 1234:
        raise Exception
    time.sleep(0.0001)

The progress bar stops at the last complete iteration, 1234.

This feature works as well with nested loops, threading, and
multiprocessing. For example, in the following code, the loops in five
threads break at 1235th iteration.

In [ ]:
def run_with_threading():
    def task(n, name):
        for i in atpbar(range(n), name=name):
            if i == 1234:
                break
            time.sleep(0.0001)
    nthreads = 5
    threads = [ ]
    for i in range(nthreads):
        name = 'thread {}'.format(i)
        n = random.randint(3000, 10000)
        t = threading.Thread(target=task, args=(n, name))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    flush()

run_with_threading()

### Progress of starting threads and processes with progress bars

`atpbar` can be used for a loop that starts sub-threads or
sub-processes in which `atpbar` is also used.

In [ ]:
def run_with_threading():
    def task(n, name):
        for i in atpbar(range(n), name=name):
            time.sleep(0.0001)
    nthreads = 5
    threads = [ ]
    for i in atpbar(range(nthreads)):
        name = 'thread {}'.format(i)
        n = random.randint(200, 1000)
        t = threading.Thread(target=task, args=(n, name))
        t.start()
        threads.append(t)
        time.sleep(0.1)
    for t in threads:
        t.join()
    flush()

run_with_threading()

The `atpbar` sensibly works regardless of the order in which multiple
instances of `atpbar` in multiple threads and multiple processes start
and end.

*****

## License

- atpbar is licensed under the BSD license.

*****

## Contact

- Tai Sakuma - tai.sakuma@gmail.com
